# Complex Analysis Deep — Contour Integration & Residues

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 169-196
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# Standard scientific stack
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import hsv_to_rgb
import sympy as sp
from scipy import integrate

%matplotlib inline

# Publication-quality defaults
plt.rcParams.update({
    "figure.figsize": (10, 7),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "lines.linewidth": 2,
    "legend.fontsize": 11,
    "font.family": "serif",
    "figure.dpi": 120,
})
print("Imports ready — numpy, matplotlib, sympy, scipy loaded.")

\
## 1. Analytic Functions and the Cauchy–Riemann Equations

A complex function $f(z) = u(x,y) + i\,v(x,y)$ is **analytic** (holomorphic) at
a point if and only if the **Cauchy–Riemann equations** hold:

$$\frac{\partial u}{\partial x} = \frac{\partial v}{\partial y}, \qquad
  \frac{\partial u}{\partial y} = -\frac{\partial v}{\partial x}$$

These conditions guarantee that $f'(z)$ exists and is independent of the
direction of approach in the complex plane.

### Example: $f(z) = z^2$

Writing $z = x + iy$:

$$f(z) = (x+iy)^2 = (x^2 - y^2) + i(2xy)$$

So $u = x^2 - y^2$, $v = 2xy$. Check:

$$u_x = 2x = v_y, \quad u_y = -2y = -v_x \;\checkmark$$

In [ ]:
\
# Verify Cauchy-Riemann equations symbolically for several functions
x, y = sp.symbols('x y', real=True)

functions = {
    "z^2":   (x**2 - y**2, 2*x*y),
    "e^z":   (sp.exp(x)*sp.cos(y), sp.exp(x)*sp.sin(y)),
    "sin(z)": (sp.sin(x)*sp.cosh(y), sp.cos(x)*sp.sinh(y)),
    "1/z":   (x/(x**2+y**2), -y/(x**2+y**2)),
}

print("Cauchy–Riemann Verification")
print("=" * 55)
for name, (u, v) in functions.items():
    ux = sp.diff(u, x)
    vy = sp.diff(v, y)
    uy = sp.diff(u, y)
    vx = sp.diff(v, x)
    cr1 = sp.simplify(ux - vy)
    cr2 = sp.simplify(uy + vx)
    status = "ANALYTIC" if cr1 == 0 and cr2 == 0 else "NOT analytic"
    print(f"\nf(z) = {name}")
    print(f"  u_x = {ux},  v_y = {vy}  →  u_x - v_y = {cr1}")
    print(f"  u_y = {uy},  v_x = {vx}  →  u_y + v_x = {cr2}")
    print(f"  ⇒ {status}")

\
## 2. Conformal Mappings

An analytic function $f(z)$ with $f'(z) \neq 0$ defines a **conformal mapping**
— it preserves angles between curves. Key examples:

| Mapping | Formula | Geometric effect |
|---------|---------|-----------------|
| Square  | $w = z^2$ | Doubles angles at origin |
| Exponential | $w = e^z$ | Horizontal strips → sectors |
| Möbius | $w = \frac{az+b}{cz+d}$ | Circles/lines → circles/lines |
| Joukowski | $w = z + 1/z$ | Circles → airfoil shapes |

Conformal mappings are essential in physics for solving Laplace's equation
$\nabla^2 \phi = 0$ in complicated geometries.

In [ ]:
\
# Conformal mapping visualization: show how z^2, exp(z), and Mobius
# transform a regular grid in the complex plane

def plot_conformal(f, title, ax, xlim=(-2,2), ylim=(-2,2), N=20):
    '''Plot image of a grid under conformal map f.'''
    # Horizontal lines
    for yv in np.linspace(ylim[0], ylim[1], N):
        xs = np.linspace(xlim[0], xlim[1], 400)
        zs = xs + 1j * yv
        ws = f(zs)
        ax.plot(ws.real, ws.imag, 'b-', lw=0.5, alpha=0.6)
    # Vertical lines
    for xv in np.linspace(xlim[0], xlim[1], N):
        ys = np.linspace(ylim[0], ylim[1], 400)
        zs = xv + 1j * ys
        ws = f(zs)
        ax.plot(ws.real, ws.imag, 'r-', lw=0.5, alpha=0.6)
    ax.set_title(title, fontsize=13)
    ax.set_xlabel("Re(w)")
    ax.set_ylabel("Im(w)")
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# w = z^2
plot_conformal(lambda z: z**2, r"$w = z^2$", axes[0],
               xlim=(-1.5, 1.5), ylim=(-1.5, 1.5))

# w = exp(z)
plot_conformal(lambda z: np.exp(z), r"$w = e^z$", axes[1],
               xlim=(-2, 2), ylim=(-np.pi, np.pi))

# Mobius: w = (z - 1)/(z + 1)
plot_conformal(lambda z: (z - 1)/(z + 1), r"$w = (z-1)/(z+1)$", axes[2],
               xlim=(-2, 2), ylim=(-2, 2))

plt.tight_layout()
plt.show()
print("Conformal maps transform the rectangular grid while preserving local angles.")

\
## 3. Contour Integration

A **contour integral** is defined as:

$$\oint_C f(z)\,dz = \int_a^b f(z(t))\,z'(t)\,dt$$

where $z(t)$ parametrizes the contour $C$ from $t=a$ to $t=b$.

### Cauchy's Integral Theorem

If $f$ is analytic on and inside a simple closed contour $C$:

$$\oint_C f(z)\,dz = 0$$

### Cauchy's Integral Formula

If $f$ is analytic inside $C$ and $z_0$ is inside $C$:

$$f(z_0) = \frac{1}{2\pi i}\oint_C \frac{f(z)}{z - z_0}\,dz$$

This extraordinary formula says that the values of an analytic function on a
closed curve completely determine its values *inside* the curve.

In [ ]:
\
# Contour integration: evaluate ∮_C z^n dz around the unit circle
# for various n, verifying Cauchy's theorem and the residue of 1/z

def contour_integrate_unit_circle(f, N=10000):
    '''Numerically integrate f(z) dz around the unit circle.'''
    t = np.linspace(0, 2*np.pi, N, endpoint=False)
    dt = 2*np.pi / N
    z = np.exp(1j * t)         # z(t) = e^{it}
    dz = 1j * np.exp(1j * t)  # dz/dt = i e^{it}
    integrand = f(z) * dz * dt
    return np.sum(integrand)

print("Contour integrals ∮_C z^n dz around the unit circle |z|=1")
print("=" * 55)
for n in range(-3, 4):
    result = contour_integrate_unit_circle(lambda z, n=n: z**n)
    expected = 2j * np.pi if n == -1 else 0
    print(f"  n = {n:+d}:  numerical = {result:.6f},  "
          f"exact = {expected:.6f},  "
          f"error = {abs(result - expected):.2e}")

print("\nKey result: Only n = -1 gives nonzero result (= 2πi).")
print("This is the fundamental residue!")

In [ ]:
\
# Cauchy's Integral Formula: f(z0) = (1/2πi) ∮ f(z)/(z-z0) dz
# Verify numerically for f(z) = sin(z), z0 = 0.3 + 0.2i

def cauchy_integral_formula(f, z0, R=1.0, N=50000):
    '''Compute f(z0) via Cauchy integral formula.
    Integrate around a circle of radius R centered at the origin.'''
    t = np.linspace(0, 2*np.pi, N, endpoint=False)
    dt = 2 * np.pi / N
    z = R * np.exp(1j * t)
    dz = 1j * R * np.exp(1j * t) * dt
    integrand = f(z) / (z - z0) * dz
    return np.sum(integrand) / (2j * np.pi)

# Test with several analytic functions
z0 = 0.3 + 0.2j

test_functions = {
    "sin(z)":  (np.sin, np.sin(z0)),
    "exp(z)":  (np.exp, np.exp(z0)),
    "z^3+1":   (lambda z: z**3 + 1, z0**3 + 1),
    "cos(z^2)":(lambda z: np.cos(z**2), np.cos(z0**2)),
}

print(f"Cauchy Integral Formula verification at z₀ = {z0}")
print("=" * 65)
for name, (func, exact) in test_functions.items():
    numerical = cauchy_integral_formula(func, z0)
    err = abs(numerical - exact)
    print(f"  f(z) = {name:12s}:  CIF = {numerical:.10f}")
    print(f"  {'':12s}   exact = {exact:.10f},  error = {err:.2e}")
print("\nCauchy's formula recovers interior values from boundary data — remarkable!")

\
## 4. Laurent Series and Singularity Classification

Near an isolated singularity $z_0$, a function has a **Laurent series**:

$$f(z) = \sum_{n=-\infty}^{\infty} a_n (z - z_0)^n$$

The **principal part** $\sum_{n=-\infty}^{-1} a_n(z-z_0)^n$ classifies the singularity:

| Type | Principal part | Example |
|------|---------------|---------|
| **Removable** | Empty (all $a_{-n}=0$) | $\frac{\sin z}{z}$ at $z=0$ |
| **Pole of order $m$** | Finite ($a_{-m}\neq 0$, $a_{-n}=0$ for $n>m$) | $\frac{1}{z^2}$ at $z=0$ |
| **Essential** | Infinite series | $e^{1/z}$ at $z=0$ |

The coefficient $a_{-1}$ is the **residue**: $\text{Res}_{z_0} f = a_{-1}$.

In [ ]:
\
# Laurent series expansion using SymPy
z = sp.Symbol('z')

functions = {
    "sin(z)/z at z=0": (sp.sin(z)/z, 0),
    "1/(z^2(z-1)) at z=0": (1/(z**2 * (z - 1)), 0),
    "exp(1/z) at z=0": (sp.exp(1/z), 0),
    "1/((z-1)(z-2)) at z=1": (1/((z-1)*(z-2)), 1),
}

print("Laurent Series Expansions")
print("=" * 65)
for name, (f_expr, z0) in functions.items():
    try:
        print(f"\nf(z) = {name}")
        series = sp.series(f_expr, z, z0, n=6)
        print(f"  Series: {series}")

        # Compute residue
        residue = sp.residue(f_expr, z, z0)
        print(f"  Residue at z={z0}: {residue}")

        # Classify singularity
        # Check leading negative power
        series_dict = series.removeO()
        if hasattr(series_dict, 'as_ordered_terms'):
            terms = series_dict.as_ordered_terms()
            neg_powers = []
            for term in terms:
                coeff, powers = term.as_coeff_mul(z - z0 if z0 != 0 else z)
                for p in powers:
                    exp = p.as_base_exp()[1] if hasattr(p, 'as_base_exp') else 0
                    if hasattr(exp, 'is_negative') and exp.is_negative:
                        neg_powers.append(exp)
        # Use SymPy's built-in singularity detection
        order = sp.singularities(f_expr, z)
        print(f"  Singularities of f: {order}")
    except (NotImplementedError, ValueError) as e:
        print(f"  {name}: {type(e).__name__} — {e}")
        print(f"  (Essential singularity — Laurent series has infinitely many terms)")


\
## 5. The Residue Theorem

The **Residue Theorem** is the crown jewel of complex analysis:

$$\oint_C f(z)\,dz = 2\pi i \sum_{k} \text{Res}_{z_k} f$$

where the sum is over all singularities $z_k$ enclosed by $C$.

### Evaluating Real Integrals

The residue theorem lets us compute difficult *real* integrals by extending
them to the complex plane:

$$\int_{-\infty}^{\infty} \frac{dx}{1+x^2} = 2\pi i \cdot \text{Res}_{z=i} \frac{1}{1+z^2}
= 2\pi i \cdot \frac{1}{2i} = \pi$$

This technique is used throughout quantum mechanics for evaluating propagators,
Green's functions, and scattering amplitudes.

In [ ]:
\
# Evaluate real integrals using the residue theorem

z = sp.Symbol('z')
x_sym = sp.Symbol('x', real=True)

# Integral 1: ∫_{-∞}^{∞} dx/(1+x^2) = π
f1 = 1/(1 + z**2)
poles_f1 = sp.solve(1 + z**2, z)
res_f1 = sum(sp.residue(f1, z, p) for p in poles_f1 if sp.im(p) > 0)
result_1 = 2 * sp.pi * sp.I * res_f1
print("Integral 1: ∫_{-∞}^{∞} dx/(1+x²)")
print(f"  Poles: {poles_f1}")
print(f"  Upper half-plane residue sum: {res_f1}")
print(f"  Result: 2πi × {res_f1} = {sp.simplify(result_1)}")
print(f"  Numerical check: {float(sp.re(result_1)):.10f} vs π = {np.pi:.10f}")

# Integral 2: ∫_{-∞}^{∞} dx/(1+x^4) = π/√2
print("\nIntegral 2: ∫_{-∞}^{∞} dx/(1+x⁴)")
f2 = 1/(1 + z**4)
poles_f2 = sp.solve(1 + z**4, z)
uhp_poles = [p for p in poles_f2 if sp.re(sp.im(p)) > 0]
res_f2 = sum(sp.residue(f2, z, p) for p in uhp_poles)
result_2 = 2 * sp.pi * sp.I * res_f2
print(f"  Poles in upper half-plane: {uhp_poles}")
print(f"  Residue sum: {sp.simplify(res_f2)}")
print(f"  Result: {sp.simplify(result_2)}")
print(f"  Numerical: {float(sp.re(sp.simplify(result_2))):.10f} vs π/√2 = {np.pi/np.sqrt(2):.10f}")

# Integral 3: ∫_{-∞}^{∞} x^2/(1+x^4) dx
print("\nIntegral 3: ∫_{-∞}^{∞} x²/(1+x⁴) dx")
f3 = z**2/(1 + z**4)
res_f3 = sum(sp.residue(f3, z, p) for p in uhp_poles)
result_3 = 2 * sp.pi * sp.I * res_f3
print(f"  Result: {sp.simplify(result_3)}")
print(f"  Numerical: {float(sp.re(sp.simplify(result_3))):.10f} vs π/√2 = {np.pi/np.sqrt(2):.10f}")

# Verify all results with scipy numerical integration
from scipy.integrate import quad
for label, func, exact in [
    ("∫dx/(1+x²)", lambda x: 1/(1+x**2), np.pi),
    ("∫dx/(1+x⁴)", lambda x: 1/(1+x**4), np.pi/np.sqrt(2)),
    ("∫x²dx/(1+x⁴)", lambda x: x**2/(1+x**4), np.pi/np.sqrt(2)),
]:
    val, err = quad(func, -np.inf, np.inf)
    print(f"\n  scipy check for {label}: {val:.10f} (error ±{err:.2e}), exact = {exact:.10f}")

In [ ]:
\
# Domain coloring: visualize complex functions via phase portraits
# Hue = argument, brightness = modulus

def domain_coloring(f, xlim=(-3, 3), ylim=(-3, 3), N=800, title=""):
    '''Create a domain coloring (phase portrait) of f(z).'''
    x = np.linspace(xlim[0], xlim[1], N)
    y = np.linspace(ylim[0], ylim[1], N)
    X, Y = np.meshgrid(x, y)
    Z = X + 1j * Y

    with np.errstate(divide='ignore', invalid='ignore'):
        W = f(Z)

    # Map argument to hue [0,1]
    H = (np.angle(W) + np.pi) / (2 * np.pi)
    # Map modulus to brightness with smooth saturation
    modulus = np.abs(W)
    S = np.ones_like(H) * 0.9
    V = 1 - 1/(1 + modulus**0.3)  # smooth brightness scaling

    HSV = np.stack([H, S, V], axis=-1)
    RGB = hsv_to_rgb(HSV)
    return RGB

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# f(z) = z^2 - 1  (two zeros)
rgb1 = domain_coloring(lambda z: z**2 - 1, title=r"$z^2-1$")
axes[0].imshow(rgb1, extent=[-3,3,-3,3], origin='lower')
axes[0].set_title(r"$f(z) = z^2 - 1$ (zeros at $\pm 1$)", fontsize=13)
axes[0].set_xlabel("Re(z)"); axes[0].set_ylabel("Im(z)")

# f(z) = 1/(z^2 + 1)  (poles at ±i)
rgb2 = domain_coloring(lambda z: 1/(z**2 + 1))
axes[1].imshow(rgb2, extent=[-3,3,-3,3], origin='lower')
axes[1].set_title(r"$f(z) = 1/(z^2+1)$ (poles at $\pm i$)", fontsize=13)
axes[1].set_xlabel("Re(z)"); axes[1].set_ylabel("Im(z)")

# f(z) = exp(1/z)  (essential singularity at 0)
rgb3 = domain_coloring(lambda z: np.exp(1/z), xlim=(-2,2), ylim=(-2,2))
axes[2].imshow(rgb3, extent=[-2,2,-2,2], origin='lower')
axes[2].set_title(r"$f(z) = e^{1/z}$ (essential sing. at 0)", fontsize=13)
axes[2].set_xlabel("Re(z)"); axes[2].set_ylabel("Im(z)")

plt.tight_layout()
plt.show()
print("Phase portraits: hue = arg(f(z)), brightness = |f(z)|")
print("Zeros appear as points where all colors meet (dark); poles are bright convergence points.")
print("Essential singularity shows wild oscillation near z=0.")

\
## 6. Quantum Mechanics Connection: Complex Analysis in Physics

Complex analysis is **indispensable** in quantum mechanics and quantum field theory:

### Propagators and Green's Functions

The free-particle propagator involves a contour integral:

$$G(E) = \lim_{\epsilon \to 0^+} \frac{1}{E - H + i\epsilon}$$

The $i\epsilon$ prescription tells us *which way to close the contour* — into the
upper or lower half-plane — giving retarded vs. advanced propagators.

### The S-Matrix and Scattering

Poles of the S-matrix in the complex energy plane correspond to:
- **Bound states** (poles on the negative real axis)
- **Resonances** (poles in the lower half-plane, $E = E_0 - i\Gamma/2$)

### Path Integrals

Feynman's path integral uses a Wick rotation $t \to -i\tau$ — a conformal
mapping from Minkowski to Euclidean spacetime:

$$\langle x_f | e^{-iHt/\hbar} | x_i \rangle = \int \mathcal{D}[x]\, e^{iS[x]/\hbar}$$

### The Residue Theorem in QFT

Loop integrals in Feynman diagrams are evaluated by residues, giving physical
quantities like decay rates and cross sections.

In [ ]:
\
# QM Application: Visualize the retarded Green's function
# G_R(E) = 1/(E - E_0 + iε) and its pole structure

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Left: Pole structure of the propagator ---
ax = axes[0]
E0_values = [1.0, 2.5, 4.0]
for E0 in E0_values:
    ax.plot(E0, 0, 'rx', markersize=12, markeredgewidth=2)
    ax.annotate(f'$E_{{{E0_values.index(E0)}}}={E0}$',
                (E0, 0), textcoords="offset points",
                xytext=(5, 10), fontsize=11)

# Show iε displacement
epsilon = 0.15
for E0 in E0_values:
    ax.plot(E0, -epsilon, 'bo', markersize=8)
    ax.annotate('', xy=(E0, -epsilon), xytext=(E0, 0),
                arrowprops=dict(arrowstyle='->', color='blue', lw=1.5))

ax.axhline(0, color='gray', lw=0.5)
ax.axvline(0, color='gray', lw=0.5)
ax.set_xlabel("Re(E)", fontsize=12)
ax.set_ylabel("Im(E)", fontsize=12)
ax.set_title("Propagator Poles: $G_R = 1/(E - E_n + i\\epsilon)$", fontsize=13)
ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-1.0, 1.0)
ax.legend(["Energy eigenvalues", "Retarded poles ($-i\\epsilon$)"],
          loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)

# --- Right: |G(E)|^2 spectral function (Lorentzian) ---
ax2 = axes[1]
E = np.linspace(-1, 6, 1000)
gamma_values = [0.05, 0.2, 0.5]
E0 = 2.5

for gamma in gamma_values:
    spectral = (1/np.pi) * (gamma/2) / ((E - E0)**2 + (gamma/2)**2)
    ax2.plot(E, spectral, label=f"$\\Gamma = {gamma}$")

ax2.set_xlabel("Energy $E$", fontsize=12)
ax2.set_ylabel("$A(E) = -\\frac{1}{\\pi}\\mathrm{Im}\\,G_R(E)$", fontsize=12)
ax2.set_title(f"Spectral Function (Lorentzian) at $E_0={E0}$", fontsize=13)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("Left: Poles of G_R shifted below real axis by iε — causality requirement.")
print("Right: Spectral function A(E) is a Lorentzian; width Γ = decay rate.")
print("In the limit Γ → 0, A(E) → δ(E - E₀) (stable state).")

In [ ]:
\
# Advanced application: compute the Fresnel-type integral
# ∫_0^∞ cos(x^2) dx = √(π/2)/2  via contour methods
# (verified numerically)

from scipy.integrate import quad

# Numerical evaluation
val_cos, _ = quad(lambda x: np.cos(x**2), 0, 1000)
val_sin, _ = quad(lambda x: np.sin(x**2), 0, 1000)
exact = np.sqrt(np.pi/2) / 2

print("Fresnel Integrals (evaluated via contour rotation z → z·e^{iπ/4})")
print("=" * 55)
print(f"  ∫₀^∞ cos(x²) dx = {val_cos:.10f}")
print(f"  ∫₀^∞ sin(x²) dx = {val_sin:.10f}")
print(f"  Exact: √(π/2)/2 = {exact:.10f}")
print(f"  Error: {abs(val_cos - exact):.2e}")

# Visualize the Fresnel spiral (Cornu spiral)
t = np.linspace(0, 8, 5000)
C = np.array([quad(lambda s: np.cos(s**2), 0, ti)[0] for ti in t])
S = np.array([quad(lambda s: np.sin(s**2), 0, ti)[0] for ti in t])

fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(C, S, 'b-', lw=1.5)
ax.plot(C[0], S[0], 'go', markersize=10, label='Start (t=0)')
ax.plot(exact, exact, 'r*', markersize=15, label=f'Limit point ({exact:.4f}, {exact:.4f})')
ax.set_xlabel("$C(t) = \\int_0^t \\cos(s^2)\\,ds$", fontsize=12)
ax.set_ylabel("$S(t) = \\int_0^t \\sin(s^2)\\,ds$", fontsize=12)
ax.set_title("Cornu Spiral (Fresnel Integrals)", fontsize=14)
ax.set_aspect('equal')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print("The Cornu spiral converges to (√(π/2)/2, √(π/2)/2) — derived via contour rotation.")

\
## Summary

| Topic | Key Result |
|-------|-----------|
| Cauchy–Riemann | $u_x = v_y$, $u_y = -v_x$ — test for analyticity |
| Conformal maps | Analytic functions with $f' \neq 0$ preserve angles |
| Cauchy's theorem | $\oint_C f\,dz = 0$ for analytic $f$ |
| Cauchy's formula | $f(z_0) = \frac{1}{2\pi i}\oint \frac{f(z)}{z-z_0}dz$ |
| Laurent series | $f(z) = \sum_{n=-\infty}^{\infty} a_n(z-z_0)^n$ |
| Residue theorem | $\oint_C f\,dz = 2\pi i \sum \text{Res}_{z_k}f$ |
| QM connection | Propagators, S-matrix poles, path integrals |

**Next:** Month 08 — Electromagnetism and Maxwell's Equations

---
*SIIEA Quantum Engineering Curriculum — CC BY-NC-SA 4.0*